# Gruppen auf endlichen Mengen
## Gruppen

### Definition Additive Gruppe

$\underline{G} = \{0, 1, 2, 3, ..., m-1\}$, wobei $m > 0$

$a \oplus b = r$

$\frac{a+b}{m}=q+\frac{r}{m} $$\iff$$ a + b = q \cdot m + r $

#### Additiv inverses Element

$a \oplus (-a) = a \oplus (m - a) = 0$
$\iff$
$\frac{a+(m-a)}{m}=1+\frac{0}{m}$

In [4]:
import math
import decimal

def gadd(m,a,b):
    return (a+b)%m

m = int (decimal.Decimal (input ("m:")))
a = int (decimal.Decimal (input ("a:")))
b = int (decimal.Decimal (input ("b:")))

print ("m =",m, ":", a, "⊕", b, "=",gadd(m,a,b))

m: 5
a: 2
b: 4


m = 5 : 2 ⊕ 4 = 1


### Definition Multiplikative Gruppe

$\underline{G} = \{0, 1, 2, 3, ..., p-1\}$, wobei $p$ eine Primzahl ist.

$a \odot b = r$

$\frac{a \cdot b}{p}=q+\frac{r}{p} $$\iff$$ a \cdot b = q \cdot p + r $

#### Multiplikativ inverses Element

$ c \odot a = a \odot c = 1 $$\iff$$ c \cdot a = q \cdot p + 1$

In [7]:
import math
import decimal

def gmul(p,a,b):
    return (a*b)%p

p = int (decimal.Decimal (input ("p:")))
a = int (decimal.Decimal (input ("a:")))
b = int (decimal.Decimal (input ("b:")))

print ("p =",p, ":", a, "⊙", b, "=",gmul(p,a,b))

p: 5
a: 2
b: 2


p = 5 : 2 ⊙ 2 = 4


### Definition Zahlenkoerper

Die Menge $\underline{F}$ sei ein Zahlenkoerper bezueglich der Verknuepfungen $\oplus\$ und $\odot$, wenn folgende Bedingungen erfuellt sind. 

  1. $\underline{F}$ ist bezueglich $\oplus$ eine additive Gruppe. Das Identitaetselement dieser Gruppe sei das Nullelement.
  2. Die von Null verschiedenen Elemente von $\underline{F}$  bilden eine multiplikative Gruppe bezueglich $\odot$. Das Identitaetselement dieser Gruppe sei das Einselement.
  3. Die Multiplication $\odot$ ist distributiv bezueglich der Addition $\oplus$, so dass gilt $a \odot (b \oplus c) = (a \odot b)\oplus(a \odot c)$ fuer $a, b, c \in \underline{F}$

### Multiplikativ Inverses Element

In [2]:
from IPython.display import Math,HTML
import math
import decimal
import pandas as pd

table = []

def mulInv(n, a, depth =  1):
    table.append({"n": n, "a": a, "q": int(n/a), "r": n%a})
    if n%a == 0:
        return depth
    return mulInv(a, n%a, depth+1)
                  
def calcXy():
    
    first = True
    lastY = 1
    for i in reversed(table):
        if first:
            i['x'] = 0
            i['y'] = 1
            first = False
        else:
            i['x'] = lastY
            i['y'] = (1-i['n']*i['x'])/i['a']
            lastY = i['y']

def getAInverse():
    row = table[0]

    display(Math(str(row['n']) + '\cdot' + str(row['x']) + '+' + str(row['a']) + '\cdot' + str(row['y']) + '=1 /mod(' + str(row['n']) + ')' ))
    display(Math('a \cdot a^{-1} = 1'))
    display(Math(str(row['a']) + '\cdot a^{-1} = 1'))
    display(Math(str(row['a']) + '\cdot' + str(row['y']) + '=1'))
    
    if (row['y'] > 0):
        return row['y']
    else:
        return row['n']+row['y']
            
def printTable():
    d = pd.DataFrame.from_dict(table)
    display(d)

    
    
n = int (decimal.Decimal (input ("n:")))
a = int (decimal.Decimal (input ("a:")))

mulInv(n,a)
calcXy()
display(Math('nx+ay=1'))
display(Math('y=\\frac{1-nx}{a}'))
printTable()
display(Math('a^{-1} =' + str(getAInverse())))


n: 7
a: 4


<IPython.core.display.Math object>

<IPython.core.display.Math object>

,n,a,q,r,x,y
0,7,4,1,3,-1.0,2.0
1,4,3,1,1,1.0,-1.0
2,3,1,3,0,0.0,1.0


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Diffie-Hellman Schlüsselaustausch

Alice und Bob vereinbaren Modul $p$ und Basis $g$

Alice:
 - Erstellt (geheim): $a$
 - Berechnet: $\alpha = g^a \mod p$
 - Sendet $\alpha$ zu Bob
 - Empängt $\beta$ von Bob
 - Berechnet: $\beta^a \mod p$

Bob:
 - Erstellt (geheim): $b$
 - Berechnet: $\beta = g^b \mod p$
 - Sendet $\beta$ zu Alice
 - Empängt $\alpha$ von Alice
 - Berechnet: $\alpha^b \mod p$
 
 $K = g^{a \cdot b} \mod p = \beta^a \mod p = \alpha^b \mod p$


In [ ]:
from IPython.display import Math,HTML
import math
import decimal

p = int (decimal.Decimal (input ("p:")))
g = int (decimal.Decimal (input ("g:")))
a = int (decimal.Decimal (input ("a:")))
b = int (decimal.Decimal (input ("b:")))

calc(g,a,p):
    return (g**a)%p

alpha = calc(g,a,p)
display(Math('\alpha = ' + str(g) + '^{' + str(a) + '} \mod ' + str(p) + ' = ' + str('alpha'))
beta = calc(g,b,p)
alpha_b = calc(alpha,b,p)
beta_a = calc(beta,a,p)

if (alpha_b == beta_a):
    



        

# Modulare reduktion

Beispiel:

$g^{a+b} = g^b \cdot g^b$

$17^{23} \mod 26$

aufsplitten funktioniert nur wenn $g^a > p$ 

$17^{23} = 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^4 \cdot 17^1 = $

$17^4 \mod 26 = 9$

$17^{23} = 9 \cdot 9 \cdot 9 \cdot 9 \cdot 9 \cdot 9 \cdot 17 = 9^3 \cdot 9^3 \cdot 17$

$9^3 \mod 26 = 1$

$17^{23} = 1 \cdot 1 \cdot 17$

In [34]:
from IPython.display import Math,HTML
import math
import decimal

def modRed(g, a, p, reduceTo):
    display(Math(str(g) +'^{' + str(a) + '} \mod ' + str(p)))
    display(Math('\iff'))
    
    np = int(a / reduceTo)
    nl = a % reduceTo

    string = ''
    for i in range(np):
        string += str(g) + '^' + str(reduceTo) + ' \mod ' + str(p) + ' \cdot '
    string += str(g) + '^{' + str(nl) + '} \mod ' + str(p)
    display(Math(string))
    
    reduced = (g**reduceTo)%p
    display(Math(str(g) + '^' + str(reduceTo) + ' \mod ' + str(p) + ' = ' + str(reduced))) 
    reduced2 = (g**nl)%p
    display(Math(str(g) + '^{' + str(nl) + '} \mod ' + str(p) + ' = ' + str(reduced2)))
    
    result = (reduced**np*reduced2)%p
    display(Math(str(reduced) + '^{' + str(np) + '} \cdot ' + str(reduced2) + ' \mod ' + str(p) + ' = ' + str(result)))
          
g = int (decimal.Decimal (input ("g:")))
a = int (decimal.Decimal (input ("a:")))
p = int (decimal.Decimal (input ("p:")))
          
modRed(g,a,p,3)
          

g: 17
a: 23
p: 26


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# GGT berechnen

In [8]:
import math
import decimal

def ggt_v01(x, y):
    while x > 0 and y > 0:
        if x >= y:
            x = x - y
        else:
            y = y - x
    return x+y

def ggt_v02(x, y):
    rest = 1
    while rest != 0: 
        if x >= y:
            rest = x % y
            x = y
            y = rest
            ggt = x
        else:
            rest = y % x
            y = x
            x = rest
            ggt=y
    return ggt


p = int (decimal.Decimal (input ("Zahl 1:")))
q = int (decimal.Decimal (input ("Zahl 2:")))

print("GGT Ansatz 1:",ggt_v01(p,q))
print("GGT Ansatz 2:",ggt_v02(p,q))


Zahl 1: 234465
Zahl 2: 214


GGT Ansatz 1: 1
GGT Ansatz 2: 1


# Eulersche $\phi$-Funktion

## Satz von Euler
Fuer eine gegebene natuerliche Zahl $n$ gilt:

$a^{\phi(n)} = a^0 = 1 \mod n$ fuer $a>0$ und $ggT(n,a)=1$

Der Satz gilt fuer alle positiven und ganzzahligen Werte von $a$, die teilerfremd zu $n$ sind.

- Der Satz von Euler kann genutzt werden um das multiplikativ inverse Element $a^{-1}$ zu berechnen.

=> auf dem Exponenten kann modulo $\phi(n)$ gerechnet werden:

$a^{j \cdot \phi(n)) \mod \phi(n)} = a^0 = 1 \mod n$



# RSA

$K_{pub}=(n,e)$

$K_{pr}=(d)$

Eine Verschluesselungsfunktion $e()$ muss umkehrbar sein => es muss eine Funktion $d()$ geben, welche die Wirkung von $e()$  rueckgaenig macht:

$d_K(y) = d_K(e_K(x)) = x$

Die Kombination von Ver- und Entschluesselung fuehrt dabei auf 

$d_K(y) = d_K(e_K(x)) = e^{e \cdot d} = x^1 = x \mod n$
## Key erstellen

In [8]:
import math

def rsa_key(p,q):
    n = p*q
    phi_n = (p-1)*(q-1)
    e = 6157
    while math.gcd(e,phi_n)>1:
         e = e+1        
    d = 0
    h = 2
    while h!=1:
          d = d + 1
          h = e * d % phi_n
    return (n,e,d,phi_n)

p = int (decimal.Decimal (input ("p:")))
q = int (decimal.Decimal (input ("q:")))

n,e,d,phi_n = rsa_key(p,q)
print("n:", n)
print("e:", e)
print("d:", d)
print("phi_n:", phi_n)

p: 17
q: 31


n: 527
e: 6157
d: 133
phi_n: 480


## Verschluesseln

$ y=e_{K_{pub}}(x)=x^e \mod n $

In [6]:
import math

def rsa_encrypt(n,e,x):
    y =x**e % n
    return y 

n = int (decimal.Decimal (input ("n (Public):")))
e = int (decimal.Decimal (input ("e (Public):")))
x = int (decimal.Decimal (input ("x (Klartext):")))

y = rsa_encrypt(n,e,x)
print ("y (Geheimtext): ", y)

n (Public): 55
e (Public): 7
x (Klartext): 3


y (Geheimtext):  42


## Entschluesseln

$ x=d_{K_{pr}}(y)=y^d \mod n $

In [7]:
import math

def rsa_decrypt(n,d,y):
    x =y**d % n
    return x 


n = int (decimal.Decimal (input ("n (Public):")))
d = int (decimal.Decimal (input ("d (Private):")))
y = int (decimal.Decimal (input ("y (Geheimtext):")))

x = rsa_encrypt(n,d,y)
print ("x (Klartext): ", x)

n (Public): 55
d (Private): 17
y (Geheimtext): 42


x (Klartext):  37
